In [ ]:
from pathlib import Path

# Create dir structure
base_img_dir = Path('mnist_images')

for sub in ['test', 'training']:
    (base_img_dir / sub).mkdir(parents=True, exist_ok=True)

weeks_dir = base_img_dir / 'weeks'
for i in range(1, 8):
    (weeks_dir / f'week {i}').mkdir(parents=True, exist_ok=True)

In [1]:
import os
import tensorflow as tf
import numpy as np
from pathlib import Path
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten
from sklearn.model_selection import train_test_split
from PIL import Image

# Get the week directory from the environment variable
week = os.getenv('WEEK', 'week 1')  # Default to 'week 1'
week_dir = Path(f'mnist_images/weeks/{week}')
train_dir = Path('mnist_images/training')
test_dir = Path('mnist_images/test')
print(f"Using week directory: {week_dir}")

train_dir.mkdir(parents=True, exist_ok=True)
test_dir.mkdir(parents=True, exist_ok=True)

image_paths = sorted(list(week_dir.glob('*.png')))
images = []
labels = []

for img_path in image_paths:
    img = Image.open(img_path).convert('L')  # Ensure grayscale
    img_array = np.array(img) / 255.0        # Normalize
    images.append(img_array)
    # Parse label from filename, for example '00001_label_7.png'
    label = int(img_path.stem.split('_')[-1])
    labels.append(label)

images = np.array(images)
labels = np.array(labels)

x_train, x_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=17, stratify=labels
)

for idx, (img, label) in enumerate(zip(x_train, y_train)):
    img_uint8 = (img * 255).astype(np.uint8)  # Convert back to uint8 for saving
    img_path = train_dir / f'{idx:05d}_label_{label}.png'
    Image.fromarray(img_uint8).save(img_path)

for idx, (img, label) in enumerate(zip(x_test, y_test)):
    img_uint8 = (img * 255).astype(np.uint8)
    img_path = test_dir / f'{idx:05d}_label_{label}.png'
    Image.fromarray(img_uint8).save(img_path)

print(f"x_train shape: {x_train.shape}, x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")

model = Sequential([
    Input(shape=(28, 28)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=2)

# Save the model
model.export('mnist_model')

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Accuracy: {test_acc * 100:.2f}%")

Using week directory: mnist_images/weeks/week 1
x_train shape: (56000, 28, 28), x_test shape: (14000, 28, 28)
y_train shape: (56000,), y_test shape: (14000,)


2025-06-11 16:49:47.933804: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-06-11 16:49:47.934430: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-06-11 16:49:47.934477: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1749649787.934953 43169457 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1749649787.935206 43169457 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/5


2025-06-11 16:49:49.434029: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1750/1750 - 21s - 12ms/step - accuracy: 0.8943 - loss: 0.3703
Epoch 2/5
1750/1750 - 20s - 11ms/step - accuracy: 0.9142 - loss: 0.3113
Epoch 3/5
1750/1750 - 19s - 11ms/step - accuracy: 0.9138 - loss: 0.3132
Epoch 4/5
1750/1750 - 18s - 11ms/step - accuracy: 0.9116 - loss: 0.3256
Epoch 5/5
1750/1750 - 21s - 12ms/step - accuracy: 0.9099 - loss: 0.3379
INFO:tensorflow:Assets written to: mnist_model/assets


INFO:tensorflow:Assets written to: mnist_model/assets


Saved artifact at 'mnist_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  6300577680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6334390800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6334390224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6334390608: TensorSpec(shape=(), dtype=tf.resource, name=None)
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9016 - loss: 0.3806
Accuracy: 90.46%
